In [1]:
import numpy as np
import random
from gurobipy import *

# The first parameter is location of the data
# Second is class column number
# The last is class identifier

def MSM(location, cc, ci):
    
    ## Pretreat data
    # Open the data
    data = np.loadtxt(location, delimiter=',', dtype = str)

    # If breast cancer data, delete id column
    if 'breast' in location:
        data = data[:,1:]

    # Delete the sample including'?'
    L = len(data) # The numer of sample
    l = len(data[0]) - 1 # Dimension of data
    x = []

    for i in range(L):
        x.append(i)
        for j in range(l):
            if data[i,j] == '?':
                x.remove(i)

    data = data[x].astype('float64') # Converse string to float64

    # Modify after remove all samples including '?'
    L = len(data)
    l = len(data[0]) - 1

    # Add 0 column to the right side for checking to classify the data
    data1 = np.c_[data,np.zeros(L)]

    # Divide the data to train set and test set by 8 : 2
    np.random.shuffle(data1)
    train = data1[:int((len(data1)) * .8)]
    test = data1[len(train):]


    smallnum = 1.e-3
    bignum = 1.e+3

    # Modify class column number after adding 0 column
    if cc < 0:
        cc -= 1

    ## Start train
    Atrain, Btrain = [], []
    apl, bpl, Cl = [], [], [] # Lists for storing ap, bp, C

    # Divide train set into class ci and the others
    for i in range(len(train)):
        if train[i, cc] == ci:
            Atrain.append(train[i])
        else :
            Btrain.append(train[i])

    Atrain, Btrain = np.array(Atrain), np.array(Btrain)

    pl=[]    # Store all p (result) by order
    rsl = [] # Store rs
    oldfuzzy = [] # Store fuzzy (the number of unclassified points)
    itera = 1 # Iteration number

    # Iteration
    while itera > 0:
        p = [0] * l # A list including only 0, the number of 0 is l
        rs = random.random() # Random seed : any number in 0 ~ 1
        random.seed(rs)
        rsl.append(rs) # Store random seed to check that the value of seed is same with the value of prior seed

        # Infinite loop for searching random values
        while True:
            for i in range(l):
                p[i] += random.uniform(-1.0, 1.0) # Store any number in -1 ~ 1 in the list p
            # If sum of p^2 >= 0.5, break the loop
            if sum(p[i] ** 2 for i in range(l)) >= 0.5:
                pl.append(p) # Store the list p
                break

        # Collect all errors
        lA, lB = len(Atrain), len(Btrain) # the number of samples of Atrain and Btrain

        # Infinite loop for the article
        while True:
            # Declare model with gurobi
            m = Model()

            # Delete the result of model optimizing
            m.setParam(GRB.Param.OutputFlag, 0)

            # Declare variables
            ap = m.addVar(vtype = GRB.CONTINUOUS, name = 'ap', ub = bignum, lb = -bignum)
            bp = m.addVar(vtype = GRB.CONTINUOUS, name = 'bp', ub = bignum, lb = -bignum)

            # Object function
            m.setObjective(ap - bp, GRB.MAXIMIZE)

            # Store all values of C in the list C
            C = []
            for i in range(l):
                C.append(m.addVar(vtype = GRB.CONTINUOUS, name = 'C[{}]'.format(i), ub = bignum, lb = -bignum))

            # A constraint
            for i in range(lA):
                if Atrain[i, -1] == 0:
                    m.addConstr(sum(C[j] * Atrain[i,j] for j in range(l)) - ap - smallnum >= 0, name = 'ctp[{}]'.format(i))

            # B constraint
            for i in range(lB):
                if Btrain[i, -1] == 0:
                    m.addConstr(sum(C[j] * Btrain[i,j] for j in range(l)) - bp + smallnum <= 0, name = 'ctq[{}]'.format(i))

            # Deletion null solution constraint
            m.addConstr(sum(p[i] * C[i] for i in range(l)) >= (.5 + sum(p[i] ** 2 for i in range(l))) / 2, name = 'null')

            # Update and Optimize model
            m.update()
            m.optimize()

            # If C converges to p, break the loop
            k = 0
            for i in range(l):
                if (abs(C[i].x - p[i]) < smallnum):
                    k += 1

            if k == l:
                break

            # If not, p = c and continue
            for i in range(l):
                p[i] = C[i].x

    #################################################################################################  

        # Convert natural number to ordinal number
        def suffix(itera):
            if (itera % 10) == 1:
                return 'st'
            elif (itera % 10) == 2:
                return 'nd'
            elif (itera % 10) == 3:
                return 'rd'
            else:
                return 'th'

        # Check C = prior C after the first iteration
        Ci = 0    
        for i in range(l):
            if itera > 1 and abs(Cl[-1][i] - C[i].x) > smallnum:
                Ci += 1

        # The first iteration
        Cll = []
        if itera == 1:
            for i in range(l):
                Cll.append(C[i].x) # Add all values of C to the list Cll

            # Store all coefficients (results)
            Cl.append(Cll) # Add Cll to the list Cl
            apl.append(ap.x) # Add all values of ap to the list apl
            bpl.append(bp.x) # Add all values of bp to the list bpl

            sA, sB = 0, 0

            # Classify all unclassified points 
            for i in range(lA):
                if (Atrain[i, -1] == 0)  and (sum(C[j].x * Atrain[i, j] for j in range(l)) - bp.x > 0):
                    Atrain[i, -1] = 1
                    sA +=1
            print('Number of classified A points at ', itera, suffix(itera), ' iteration is : ', sA, sep = '')

            for i in range(lB):
                if (Btrain[i, -1] == 0)  and (sum(C[j].x * Btrain[i, j] for j in range(l)) - ap.x > 0):
                    Btrain[i, -1] = 1
                    sB +=1
            print('Number of classified B points at ', itera, suffix(itera), ' iteration is : ', sB, sep = '')

            # Check the number of unclassified points         
            fuzzy = 0
            for i in range(lA):
                if Atrain[i, -1] == 0:
                    fuzzy += 1

            for i in range(lB):
                if Btrain[i, -1] == 0:
                    fuzzy += 1

            # Complete classification
            if ap.x >= bp.x or fuzzy == 0:
                itera = 0
                print('-----')
                print('Successful termination with 0 misclassification!')
                print('-----')

            # If not, the number of unclassified points is added to the list oldfuzzy and continue
            else:
                oldfuzzy.append(fuzzy)
                itera += 1

        # After the first iteration
        elif itera > 1:

            if Ci == l and abs(apl[-1] - ap.x) > smallnum and abs(bpl[-1] - bp.x) > smallnum:

                for i in range(l):
                    Cll.append(C[i].x) # If C != prior C, add all values of C to the list Cll

                # Store all coefficients (results)
                Cl.append(Cll) # Add Cll to the list Cl
                apl.append(ap.x) # Add all values of ap to the list apl
                bpl.append(bp.x) # Add all values of bp to the list bpl

                # Classify all unclassified points outside 2 planes
                sA,sB = 0, 0

                for i in range(lA):
                    if(Atrain[i,-1] == 0) and (sum(C[j].x * Atrain[i,j] for j in range(l)) - bp.x > 0):
                        Atrain[i,-1] = 1
                        sA += 1
                print('Number of classified A points at ', itera, suffix(itera), ' iteration is : ', sA, sep = '')

                for i in range(lB):
                    if(Btrain[i,-1] == 0) and (sum(C[j].x * Btrain[i,j] for j in range(l)) - ap.x > 0):
                        Btrain[i,-1] = 1
                        sB += 1
                print('Number of classified B points at ', itera, suffix(itera), ' iteration is : ', sB, sep = '')

                # Check the number of unclassified points   
                fuzzy = 0
                for i in range(lA):
                    if Atrain[i, -1] == 0:
                        fuzzy += 1

                for i in range(lB):
                    if Btrain[i, -1] == 0:
                        fuzzy += 1

                # Complete classification
                if ap.x >= bp.x or fuzzy == 0:
                    itera = 0
                    print('-----')
                    print('Successful termination with 0 misclassification!')
                    print('-----')

                # If too many iterations, fail classification
                elif itera > 3 and oldfuzzy[-2] == oldfuzzy[-1] == fuzzy :
                    itera = 0
                    print('------')
                    print('Fail to classify : termination with' , fuzzy, 'misclassifications')
                    print('------')

                # If not, the number of unclassified points is added to the list oldfuzzy and continue
                else :
                    oldfuzzy.append(fuzzy)
                    print('Number of unclassified points at ', itera, suffix(itera), ' iteration is : ', oldfuzzy[-1], sep = '')
                    itera += 1

    ## Start test
    Atest, Btest = [], []

    Cl = np.array(Cl)

    fail = 0

    for i in range(len(test)):
        if test[i,cc] == ci:
            Atest.append(test[i])
        else :
            Btest.append(test[i])

    Atest, Btest = np.array(Atest), np.array(Btest)
    lA, lB, c = len(Atest), len(Btest), len(Cl) # c is iteration number

    ptest, qtest = np.zeros((c,lA)), np.zeros((c,lB))

    for i in range(c):
        for j in range(lA):
            for k in range(l):
                ptest[i,j] += Cl[i,k] * Atest[j,k]

    for i in range(c):
        for j in range(lB):
            for k in range(l):
                qtest[i,j] += Cl[i,k] * Btest[j,k]

    w = 0
    while w < c: # iteration
        lA, lB = len(Atest), len(Btest)
        rmpostest, rmnegtest = [], []

        for i in range(lA):
            if(ptest[w,i] - apl[w] < 0):
                fail += 1
        for i in range(lB):
            if(qtest[w,i] - bpl[w] > 0):
                fail += 1

        for i in range(lA):
            if(ptest[w,i] - bpl[w] < 0):
                rmpostest.append(i)

        for i in range(lB):
            if(qtest[w,i] - apl[w] > 0):
                rmnegtest.append(i)

        Atest, Btest = Atest[rmpostest], Btest[rmnegtest]

        w += 1

    success_rate = (len(test) - fail) * 100 / len(test)
    print('The success rate of classification by class identifier', ci, 'and the others:', success_rate, '%')
    print()
    print()

# Classify breast cancer
location = 'data/breast-cancer-wisconsin.data'
MSM(location, -1, 2)
MSM(location, -1, 4)

# Classify wine
location = 'data/wine.data'
MSM(location, 0, 1)
MSM(location, 0, 2)
MSM(location, 0, 3)

# Classify heart disease
location = 'data/processed.cleveland.data'
MSM(location, -1, 0)
MSM(location, -1, 1)
MSM(location, -1, 2)
MSM(location, -1, 3)
MSM(location, -1, 4)

Academic license - for non-commercial use only
Number of classified A points at 1st iteration is : 239
Number of classified B points at 1st iteration is : 100
Number of classified A points at 2nd iteration is : 115
Number of classified B points at 2nd iteration is : 0
-----
Successful termination with 0 misclassification!
-----
The success rate of classification by class identifier 2 and the others: 91.24087591240875 %


Number of classified A points at 1st iteration is : 142
Number of classified B points at 1st iteration is : 59
Number of classified A points at 2nd iteration is : 45
Number of classified B points at 2nd iteration is : 0
-----
Successful termination with 0 misclassification!
-----
The success rate of classification by class identifier 4 and the others: 95.62043795620438 %


Number of classified A points at 1st iteration is : 46
Number of classified B points at 1st iteration is : 0
-----
Successful termination with 0 misclassification!
-----
The success rate of classific